# Collaborative Filtering

escolha 3 utilizadores e faça a sugestão de um top 5 de anedotas por utilizador. Tenham em consideração que os utilizadores escolhidos devem ter gostos por tipos de anedotas diferentes. Justifiquem a escolha dos utilizadores.
Desta maneira será possível avaliar se as anedotas sugeridas realmente foram bem-sucedidas.
Adicionalmente, descrevam a abordagem escolhida para recomendação de anedotas e justifiquem o porquê dessa abordagem ser a melhor neste contexto.

In [1]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

# import data
ratings = pd.read_csv("CF/ratings.csv", index_col=0)
ratings

,jokeId,rating
userId,,
1,5,0.219
1,7,-9.281
1,8,-9.281
1,13,-6.781
1,15,0.875
...,...,...
63978,57,-8.531
63978,24,-9.062
63978,124,-9.031


In [2]:
df = pd.DataFrame(ratings)

In [3]:
# Usando pivot para transformar a tabela
pivot_table = df.pivot(columns='jokeId', values='rating')

print(pivot_table)

jokeId    5      7      8      13     15     16     17     18     19     20   \
userId                                                                         
1       0.219 -9.281 -9.281 -6.781  0.875 -9.656 -9.031 -7.469 -8.719 -9.156   
2      -9.688  9.938  9.531  9.938  0.406  3.719  9.656 -2.688 -9.562 -9.125   
3      -9.844 -9.844 -7.219 -2.031 -9.938 -9.969 -9.875 -9.812 -9.781 -6.844   
4      -5.812 -4.500 -4.906    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5       6.906  4.750 -5.906 -0.406 -4.031  3.875  6.219  5.656  6.094  5.406   
...       ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
63974     NaN -4.438  1.531 -1.438 -9.156 -7.438  2.250  3.750  5.000    NaN   
63975     NaN  0.625  4.531  4.969 -3.375 -8.250 -7.781 -9.188  1.500    NaN   
63976     NaN -7.250  4.562 -5.594    NaN    NaN    NaN    NaN    NaN    NaN   
63977     NaN -8.531 -8.438 -9.625  6.594 -6.250  9.312 -2.750  6.344    NaN   
63978     NaN -7.906 -7.594 -7.594 -6.37

In [4]:
ratings = pivot_table.fillna(0)
ratings

jokeId,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
userId,,,,,,,,,,,,,,,,,,,,,
1,0.219,-9.281,-9.281,-6.781,0.875,-9.656,-9.031,-7.469,-8.719,-9.156,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
2,-9.688,9.938,9.531,9.938,0.406,3.719,9.656,-2.688,-9.562,-9.125,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
3,-9.844,-9.844,-7.219,-2.031,-9.938,-9.969,-9.875,-9.812,-9.781,-6.844,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
4,-5.812,-4.500,-4.906,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
5,6.906,4.750,-5.906,-0.406,-4.031,3.875,6.219,5.656,6.094,5.406,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63974,0.000,-4.438,1.531,-1.438,-9.156,-7.438,2.250,3.750,5.000,0.000,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
63975,0.000,0.625,4.531,4.969,-3.375,-8.250,-7.781,-9.188,1.500,0.000,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000
63976,0.000,-7.250,4.562,-5.594,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000


In [5]:
# standardize or normalize data
def standardize(row):
    return (row - row.mean()) / (row.max() - row.min())

ratings_std = ratings.apply(standardize)
ratings_std

jokeId,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
userId,,,,,,,,,,,,,,,,,,,,,
1,0.011950,-0.373604,-0.431252,-0.310936,0.107714,-0.410133,-0.482949,-0.368065,-0.469175,-0.457469,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
2,-0.484169,0.587346,0.509348,0.525014,0.084264,0.258617,0.451401,-0.129015,-0.511325,-0.455919,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
3,-0.491981,-0.401754,-0.328152,-0.073436,-0.432936,-0.425783,-0.525149,-0.485215,-0.522275,-0.341869,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
4,-0.290068,-0.134554,-0.212502,0.028114,0.063964,0.072667,-0.031399,0.005385,-0.033225,0.000331,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
5,0.346819,0.327946,-0.262502,0.007814,-0.137586,0.266417,0.279551,0.288185,0.271475,0.270631,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63974,0.000983,-0.131454,0.109348,-0.043786,-0.393836,-0.299233,0.081101,0.192885,0.216775,0.000331,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
63975,0.000983,0.121696,0.259348,0.276564,-0.104786,-0.339833,-0.420449,-0.454015,0.041775,0.000331,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187
63976,0.000983,-0.272054,0.260898,-0.251586,0.063964,0.072667,-0.031399,0.005385,-0.033225,0.000331,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187


In [6]:
selected_userIds = [2]
df = ratings_std[ratings.index.isin(selected_userIds)]

df.head()

jokeId,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
userId,,,,,,,,,,,,,,,,,,,,,
2,-0.484169,0.587346,0.509348,0.525014,0.084264,0.258617,0.451401,-0.129015,-0.511325,-0.455919,...,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187


# Item-based Collaborative Filtering

Foram escolhidos esses usuários por apresentarem diferentes ratins para o filmes 

# User 2

In [7]:
# Supondo que queremos encontrar o maior valor para userId=2
row_index = 2
max_value = df.loc[row_index].max()
max_column = df.loc[row_index].idxmax()

print("Maior valor na linha", row_index, ":", max_value)
print("Coluna correspondente:", max_column)

Maior valor na linha 2 : 0.5873462000270591
Coluna correspondente: 7


In [10]:
item_similarity = cosine_similarity(df.T)
item_similarity_df = pd.DataFrame(item_similarity, index=df.columns, columns=df.columns)
item_similarity_df

jokeId,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
jokeId,,,,,,,,,,,,,,,,,,,,,
5,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
8,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
13,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
15,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
147,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
148,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,...,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [11]:
def get_similar_item(name, rating):
    similar_score = item_similarity_df[name]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

result = get_similar_item(7, 5)

temp_result = []
for key, value in result.items():
    temp_result.append([key, value])
    
# Remove all the rated movies from user - beautify
temp_result = pd.DataFrame(temp_result, columns=["movie", "predicted score"])
temp_result[~temp_result['movie'].isin([7])]

,movie,predicted score
0,124,2.5
1,58,2.5
2,89,2.5
3,35,2.5
4,36,2.5
...,...,...
135,71,-2.5
136,70,-2.5
137,69,-2.5
138,68,-2.5


Sugestão de um top 5 de anedotas por utilizador.

Considerando o User 2, verifica-se a correspondencia de preferência dos 'movies' , 124, 58, 89, 35, 36. Podemos assim sugerir essas anedotas.

In [12]:
df2 = pd.read_csv('CF/items.csv')

In [13]:
# Configurar para mostrar todas as linhas e colunas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [14]:
# Buscar as linhas onde o 'jokeId' é igual a 124, 58, 89, 35, 36
linhas_selecionadas = df2.loc[df2['jokeId'].isin([124, 58, 89, 35, 36])]

# Imprimir o texto correspondente a cada jokeId
for index, row in linhas_selecionadas.iterrows():
    print(f"jokeId: {row['jokeId']}, jokeText: {row['jokeText']}")

jokeId: 35, jokeText: An explorer in the deepest Amazon suddenly finds himself surrounded
by a bloodthirsty group of natives. Upon surveying the situation, he
says quietly to himself, "Oh God, I'm screwed." 

The sky darkens and a voice booms out, "No, you are NOT
screwed. Pick up that stone at your feet and bash in the head of the
chief standing in front of you." 

So with the stone he bashes the life out of the chief. Standing above
the lifeless body, breathing heavily looking at 100 angry natives... 

The voice booms out again, "Okay ..... NOW you're screwed." 

jokeId: 36, jokeText: A guy walks into a bar, orders a beer and says to the bartender,
"Hey, I got this great Polish Joke..." 

The barkeep glares at him and says in a warning tone of voice:
"Before you go telling that joke you better know that I'm Polish, both
bouncers are Polish and so are most of my customers"

"Okay" says the customer,"I'll tell it very slowly." 

jokeId: 58, jokeText: How many teddybears does it take to

# User 103

In [15]:
selected_userIds = [103]
df = ratings_std[ratings_std.index.isin(selected_userIds)]

df.head()

jokeId,5,7,8,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
103,0.152768,-0.186104,-0.301602,0.232814,-0.054786,-0.246083,0.037351,-0.261815,0.113675,-0.304369,-0.210893,-0.007999,-0.006139,0.004106,-0.008578,-0.019952,-0.487619,-0.017921,0.187911,-0.004761,0.122006,0.002015,0.000717,-0.005612,0.183513,0.072705,-0.00084,-0.012933,-0.012957,-0.009009,-0.000474,-0.012201,-0.000043,0.004051,-0.00961,-0.010731,-0.038118,-0.02551,0.096656,0.196492,0.000063,-0.000013,0.068042,0.173718,-0.002119,-0.025842,0.002077,0.005675,-0.006482,-0.005482,-0.000362,-0.046588,-0.03312,0.000344,0.194716,0.134077,0.000169,-0.044767,0.240592,-0.011197,-0.004701,0.05493,-0.000254,0.00236,0.003871,-0.038542,-0.009442,-0.017484,-0.001234,-0.000318,-0.016581,-0.007775,-0.019052,-0.013628,-0.001222,-0.008985,-0.024402,-0.018315,-0.566776,-0.008806,-0.017979,-0.018079,-0.034348,-0.013846,-0.008588,-0.019873,-0.035249,-0.016431,-0.012751,-0.000148,0.001039,-0.006822,-0.004117,-0.057285,-0.068479,-0.062808,-0.024636,-0.046088,-0.017066,-0.024836,-0.033993,-0.02173,-0.010916,0.10102,-0.144086,-0.134729,0.209473,0.230536,-0.04273,-0.015221,-0.025928,-0.021055,0.001008,0.007746,-0.025735,-0.044529,-0.054733,-0.022471,-0.059114,-0.013656,-0.004155,-0.044428,-0.011155,-0.042379,-0.017319,-0.002251,-0.012285,-0.039836,-0.01532,-0.002936,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187


In [16]:
# Supondo que queremos encontrar o maior valor para userId=3
row_index = 103
max_value = df.loc[row_index].max()
max_column = df.loc[row_index].idxmax()

print("Maior valor na linha", row_index, ":", max_value)
print("Coluna correspondente:", max_column)

Maior valor na linha 103 : 0.24059180815801923
Coluna correspondente: 69


In [19]:
item_similarity = cosine_similarity(df.T)
item_similarity_df = pd.DataFrame(item_similarity, index=df.columns, columns=df.columns)
item_similarity_df

jokeId,5,7,8,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
jokeId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
13,1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
15,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
16,-1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.

In [20]:
def get_similar_item(name, rating):
    similar_score = item_similarity_df[name]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

result = get_similar_item(69, 5)

temp_result = []
for key, value in result.items():
    temp_result.append([key, value])
    
# Remove all the rated movies from user - beautify
temp_result = pd.DataFrame(temp_result, columns=["movie", "predicted score"])
temp_result[~temp_result['movie'].isin([69])]

,movie,predicted score
0,5,2.5
1,35,2.5
2,44,2.5
3,49,2.5
4,50,2.5
5,51,2.5
6,53,2.5
7,54,2.5
8,123,2.5
9,57,2.5


movies: 5, 98, 113, 112, 111

In [21]:
linhas_selecionadas = df2.loc[df2['jokeId'].isin([5, 98, 113, 112, 111])]

# Imprimir o texto correspondente a cada jokeId
for index, row in linhas_selecionadas.iterrows():
    print(f"jokeId: {row['jokeId']}, jokeText: {row['jokeText']}")

jokeId: 5, jokeText: Q.	What's O. J. Simpson's Internet address? 
A.	Slash, slash, backslash, slash, slash, escape.

jokeId: 98, jokeText: Age and Womanhood

1. Between the ages of 13 and 18 ...
She is like Africa, virgin and unexplored. 

2. Between the ages of 19 and 35 ...
She is like Asia, hot and exotic. 

3. Between the ages of 36 and 45 ...
She is like America, fully explored, breathtakingly beautiful,and free with her resources.

4. Between the ages of 46 and 56 ...
She is like Europe, exhausted but still has points of interest. 

5. After 56 she is like Australia ...
Everybody knows it's down there, but who gives a damn?

jokeId: 111, jokeText: A new business was opening and one of the owner's friends wanted to send him flowers for the occasion. They arrived at the new business site and the owner read the card, "Rest in Peace." The owner was angry and called the florist to complain. After he had told the florist of the obvious mistake and how angry he was, the florist replied,

# User 63974

In [22]:
selected_userIds = [63974]
df3 = ratings_std[ratings_std.index.isin(selected_userIds)]

df3.head()

jokeId,5,7,8,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
63974,0.000983,-0.131454,0.109348,-0.043786,-0.393836,-0.299233,0.081101,0.192885,0.216775,0.000331,-0.029643,-0.007999,-0.006139,0.004106,-0.008578,-0.019952,-0.000914,-0.017921,-0.037089,-0.004761,-0.000464,-0.066735,0.000717,-0.005612,-0.061787,-0.039795,-0.00084,-0.012933,-0.012957,-0.009009,-0.000474,-0.012201,-0.000043,0.004051,-0.00961,-0.010731,-0.038118,-0.02551,-0.037744,-0.045708,0.000063,-0.000013,-0.072558,-0.041882,-0.002119,-0.025842,0.002077,0.005675,-0.006482,-0.005482,-0.000362,-0.046588,-0.03312,0.000344,-0.022484,-0.037823,0.000169,-0.044767,-0.046908,-0.011197,-0.004701,-0.05757,-0.000254,0.00236,0.003871,-0.038542,-0.009442,-0.017484,-0.001234,-0.000318,-0.016581,-0.007775,-0.019052,-0.013628,-0.001222,-0.008985,-0.024402,-0.018315,0.330124,-0.008806,-0.017979,-0.018079,-0.034348,-0.013846,-0.008588,-0.019873,-0.035249,-0.016431,-0.012751,-0.000148,0.001039,-0.006822,-0.004117,-0.057285,-0.068479,-0.062808,-0.024636,0.055462,-0.017066,-0.024836,-0.033993,-0.02173,-0.010916,-0.06928,-0.012836,-0.000248,-0.049927,-0.019464,-0.04273,-0.015221,-0.025928,-0.021055,0.001008,0.007746,-0.025735,-0.044529,-0.054733,-0.022471,-0.059114,-0.013656,-0.004155,-0.044428,-0.011155,-0.042379,-0.017319,-0.002251,-0.012285,-0.039836,-0.01532,-0.002936,0.00947,-0.015341,-0.033531,-0.015412,-0.026066,-0.000741,-0.01467,-0.038803,-0.022258,-0.043187


In [23]:
# Supondo que queremos encontrar o maior valor para userId=5
row_index = 63974
max_value = df3.loc[row_index].max()
max_column = df3.loc[row_index].idxmax()

print("Maior valor na linha", row_index, ":", max_value)
print("Coluna correspondente:", max_column)

Maior valor na linha 63974 : 0.33012420855036223
Coluna correspondente: 89


In [25]:
item_similarity = cosine_similarity(df3.T)
item_similarity_df = pd.DataFrame(item_similarity, index=df3.columns, columns=df3.columns)
item_similarity_df

jokeId,5,7,8,13,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150
jokeId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
7,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
13,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
15,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
16,-1.0,1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,-1.0,1.0,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,1.0,1.0,1.0,1.0,1.0

In [26]:
def get_similar_item(name, rating):
    similar_score = item_similarity_df[name]*(rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    return similar_score

result = get_similar_item(89, 5)

temp_result = []
for key, value in result.items():
    temp_result.append([key, value])
    
# Remove all the rated movies from user - beautify
temp_result = pd.DataFrame(temp_result, columns=["movie", "predicted score"])
temp_result[~temp_result['movie'].isin([89])]

,movie,predicted score
0,5,2.5
1,141,2.5
2,58,2.5
3,51,2.5
5,123,2.5
6,124,2.5
7,44,2.5
8,64,2.5
9,108,2.5
10,67,2.5


In [27]:
# movies: 5, 141, 58, 51, 123

linhas_selecionadas = df2.loc[df2['jokeId'].isin([5, 141, 58, 51, 123])]

# Imprimir o texto correspondente a cada jokeId
for index, row in linhas_selecionadas.iterrows():
    print(f"jokeId: {row['jokeId']}, jokeText: {row['jokeText']}")

jokeId: 5, jokeText: Q.	What's O. J. Simpson's Internet address? 
A.	Slash, slash, backslash, slash, slash, escape.

jokeId: 51, jokeText: Did you hear that Clinton has announced there is a new national bird?  
The spread eagle.

jokeId: 58, jokeText: How many teddybears does it take to change a lightbulb?

It takes only one teddybear, but it takes a whole lot of lightbulbs.

jokeId: 123, jokeText: When most people claim to be "killing time", it's only an expression. When Chuck Norris kills time, the minutes actually cease to exist.
jokeId: 141, jokeText: Jack Bauer can get McDonald's breakfast after 10:30.


Foram sugeridas 5 anedotas correspondetes aos 3 Users escolhidos